# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load the csv 
output_file = "../cities_output_student.csv"

# Create DataFrame for output data file from cities/weather (Part I)
cities_df = pd.read_csv(r'../cities_output_student.csv')
cities_df.head()


,Cities,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,Muros,42.7762,-9.0603,64.99,82,17,8.70,ES,1627433403
1,Kapaa,22.0752,-159.3190,85.98,75,90,3.00,US,1627433065
2,Albany,42.6001,-73.9662,68.81,91,100,1.52,US,1627432925
3,New Norfolk,-42.7826,147.0587,58.50,69,94,4.00,AU,1627433079
4,Punta Arenas,-53.1500,-70.9167,42.91,87,75,17.27,CL,1627432891


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [23]:
# Configure gmaps
gmaps.configure(api_key = g_key)

# Store lat and lng as locations
locations = cities_df[['Latitude','Longitude']]

# Store humidity as weight
weight = cities_df['Humidity'].astype(float)

# Plot Heatmap
fig =gmaps.figure()

# Creat heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = weight, dissipating = False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [16]:
# Creat new DataFrame fitting ideal conditions
vacation_spot_df = cities_df.loc[(cities_df['Max Temperature'] > 70) & (cities_df['Max Temperature'] < 80) & (cities_df['Wind Speed'] < 10) & (cities_df['Cloudiness'] < 5)].dropna()
vacation_spot_df

,Cities,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
10,Jamestown,42.0970,-79.2353,76.32,78,1,5.75,US,1627433007
59,Mushie,-3.0167,16.9000,70.95,30,4,3.40,CD,1627433412
62,Caravelas,-17.7125,-39.2481,72.45,83,0,8.81,BR,1627433413
109,Piaçabuçu,-10.4056,-36.4344,73.13,88,0,5.19,BR,1627433421
113,Tataouine,32.9297,10.4518,79.68,74,0,7.47,TN,1627433422
125,Mitsamiouli,-11.3847,43.2844,75.78,67,0,6.11,KM,1627433221
166,Vila Velha,-20.3297,-40.2925,71.55,88,0,8.05,BR,1627433413
187,Dryden,49.7833,-92.7503,79.09,36,1,6.91,CA,1627433437
232,Charters Towers,-20.1000,146.2667,73.69,50,4,8.68,AU,1627433447
239,Changji,44.0167,87.3167,79.59,29,0,6.71,CN,1627433449


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [17]:
# Store variable named hotel_df
hotel_df = vacation_spot_df

# Add Hotel Name column
hotel_df["Hotel Name"] = ""

# Define parameters
params = {
    "radius": 5000,
    "types" : "hotel",
    "keyword" : "hotels",
    "key" : g_key
}

# Loop through API
for index, row in hotel_df.iterrows():
    
    # Define latitudes and longitudes
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    # Maintain parameters with various locations
    params["location"] = f"{lat},{lng}"
    
    # Define url
    main_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Capture hotel name
    hotel_name = requests.get(main_url, params = params)
    
    # Convert to json file
    hotel_name = hotel_name.json()
#     print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Include option to skip missing data
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing result.... Skipping.")
        
hotel_df

Missing result.... Skipping.


,Cities,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
10,Jamestown,42.0970,-79.2353,76.32,78,1,5.75,US,1627433007,Chautauqua Harbor Hotel
59,Mushie,-3.0167,16.9000,70.95,30,4,3.40,CD,1627433412,
62,Caravelas,-17.7125,-39.2481,72.45,83,0,8.81,BR,1627433413,Pousada Liberdade
109,Piaçabuçu,-10.4056,-36.4344,73.13,88,0,5.19,BR,1627433421,Hotel Cara Peba
113,Tataouine,32.9297,10.4518,79.68,74,0,7.47,TN,1627433422,Auberge Alferdaous
125,Mitsamiouli,-11.3847,43.2844,75.78,67,0,6.11,KM,1627433221,Al Camar Lodge
166,Vila Velha,-20.3297,-40.2925,71.55,88,0,8.05,BR,1627433413,Golden Tulip Porto Vitória
187,Dryden,49.7833,-92.7503,79.09,36,1,6.91,CA,1627433437,Comfort Inn
232,Charters Towers,-20.1000,146.2667,73.69,50,4,8.68,AU,1627433447,Commercial Hotel
239,Changji,44.0167,87.3167,79.59,29,0,6.71,CN,1627433449,Garden Hotel


In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{Cities}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [20]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))